<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/ScreenTime_procedure_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides the codes to check the progress of each participant, including two main functions:
1. Check the last update of screen events from each participant
2. Generate a list of emails (to be copied), to whom we should send a reminder email

# Preparation

Mount Google Drive and set up file path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the Google Drive path, depending who is running the notebook

In [ ]:
#Jian
google_drive_data_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Data'

In [ ]:
#Subin
google_drive_data_path = '/content/drive/MyDrive/UMD_research/Problematic_Smartphone_Usage'

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

# Paricipant management

## Progress check

Which day is each participant on, the state of each survey and the tutorial: unavailable, available, or completed.

### Read in the meta data

In [ ]:
# read in the data
survey_meta0 = pd.read_csv(google_drive_data_path + '/survey_meta.csv')

survey_data = survey_meta0.copy()
survey_data['submitDate'] = survey_data['timestamp'].apply(lambda x: convert_unix_to_date(x))

# Pivot the dataframe to create survey columns
survey_meta_pivoted = survey_data.pivot(index=['pid','host'], columns='surveyID', values='submitDate')

# Rename the columns
survey_meta_pivoted.columns = ['survey' + str(col) for col in survey_meta_pivoted.columns]

# Reset the index to make pid a column again
survey_meta_pivoted = survey_meta_pivoted.reset_index()

# Get today's date
today = pd.Timestamp.now().date()

# Convert survey1 to datetime if it's not already
survey_meta_pivoted['survey1'] = pd.to_datetime(survey_meta_pivoted['survey1']).dt.date

# Convert to numpy array to handle the date subtraction properly
survey_meta_pivoted['app_day_count'] = np.array([today - d for d in survey_meta_pivoted['survey1']], dtype='timedelta64[D]').astype(int) + 1

print("The shape of the dataframe is: ", survey_meta_pivoted.shape)

survey_meta_pivoted.head()

The shape of the dataframe is:  (7, 7)


,pid,host,survey1,survey2,survey3,survey4,app_day_count
0,pi001,NaN,2024-11-12,2024-11-15,NaN,NaN,15
1,pi002,NaN,2024-11-13,2024-11-15,NaN,NaN,14
2,pi003,NaN,2024-11-12,2024-11-15,2024-11-17,NaN,15
3,pi004,NaN,2024-11-14,2024-11-17,2024-11-20,NaN,13
4,pi005,NaN,2024-11-15,NaN,NaN,NaN,12


In [ ]:
# Save to google drive
survey_meta_pivoted.to_csv(google_drive_data_path + '/survey_meta_pivoted.csv', index=False)

### Read the email info and merge

Before running the cell below, make sure the Participants.xlsx is up to date.

In [ ]:
participant_data = pd.read_excel(google_drive_data_path + '/Participants.xlsx')
participant_data

,pid,email
0,pi001,pi001@gmail.com
1,pi002,pi002@gmail.com
2,pi003,pi003@gmail.com
3,pi004,pi004@gmail.com
4,pi005,pi005@gmail.com
5,te001,te001@gmail.com
6,te003,te003@gmail.com


In [ ]:
survey_meta_manager0 = pd.merge(survey_meta_pivoted, participant_data, on='pid', how='left')
survey_meta_manager = survey_meta_manager0.copy()

## Generate reminder mail list

Generate a list of emails for certain filter. Copy the output into bcc in gmail

Locate the corresponding email in the document [Emails](https://docs.google.com/document/d/1e5TZJ6ILpku8rVdO_ADUwI-lM8SCIh7xG95jrG73INY/edit?usp=sharing)

In [ ]:
def get_emails_by_day_count(day_count):
    emails = survey_meta_manager[survey_meta_manager['day_count'] == day_count]['email'].tolist()
    if emails:
        return ','.join(emails)
    else:
        return None

### Week 1 completion

In [ ]:
# filter those day_count is 7
emails = get_emails_by_day_count(7)
print(emails)

None


### Week 2 completion

In [ ]:
# filter those day_count is 14
emails = get_emails_by_day_count(14)
print(emails)

pi002@gmail.com


### Week 3 completion

In [ ]:
# filter those day_count is 21
emails = get_emails_by_day_count(21)
print(emails)

None


### Daily reminder

In [ ]:
# if there is an available survey awaiting
# Create four columns with initial 'unavailable' status
survey_meta_manager['survey1_status'] = 'unavailable'
survey_meta_manager['survey2_status'] = 'unavailable'
survey_meta_manager['survey3_status'] = 'unavailable'
survey_meta_manager['survey4_status'] = 'unavailable'

# Update status to 'complete' if there's a date in the survey columns
for i in range(1, 5):
    survey_meta_manager[f'survey{i}_status'] = np.where(survey_meta_manager[f'survey{i}'].notna(), 'complete', survey_meta_manager[f'survey{i}_status'])

# Update status to 'available' if survey is NA and day_count exceeds the threshold
thresholds = [6, 13, 20]
for i, col in enumerate(['survey2', 'survey3', 'survey4'], start=2):
    survey_meta_manager[f'{col}_status'] = np.where(
        (survey_meta_manager[col].isna()) & (survey_meta_manager['day_count'] > thresholds[i-2]),
        'available',
        survey_meta_manager[f'{col}_status']
    )

# Save the updated DataFrame
print("The shape of the dataframe is: ", survey_meta_manager.shape)
survey_meta_manager.to_csv(google_drive_data_path + '/survey_meta_manager.csv', index=False)

In [ ]:
# Get emails for participants with at least one 'available' survey
target_participants = survey_meta_manager[(survey_meta_manager['survey2_status'] == 'available') |
                                          (survey_meta_manager['survey3_status'] == 'available') |
                                          (survey_meta_manager['survey4_status'] == 'available')]
email_list = target_participants['email'].tolist()
print(','.join(email_list) or 'None')

pi001@gmail.com,pi002@gmail.com,pi005@gmail.com


### Drop out notification

In [ ]:
# If a survey is unanswered for seven days, send out a drop-out notification

## Check latest update